In [6]:
# Parâmetros do NewHope

n = 1024
q = 12289

if n != 2^(log(n,2)) or not is_prime(q) or q < n or q%(2*n) != 1:
    raise ValueError(f"os parâmetros n={n} e q={q} não verificam as condições do esquema NewHope")
    
# noise
k = 8

In [7]:
## Os anéis usuais com o módulo ciclotómico

R_.<w> = ZZ[]
R.<x>  = QuotientRing(R_,R_.ideal(w^n+1))

Rq_.<w> = GF(q)[]
Rq.<x>  = QuotientRing(Rq_,Rq_.ideal(w^n+1))

In [8]:
# Geradores aleatorios

import random, os

random.seed(os.urandom(8))         # gerar seeds diferentes para cada execução

def random_pol(n=n):
    p = R_.random_element(n)
    return Rq(p)

def chi(k=k):
    samples = range(2*k)
    return sum([random.choice([-1,1]) for _ in samples])


def random_err(n=n):
    return Rq([chi() for _ in range(n)])

In [9]:
## messages

d = GF(q)((q-1)//2)      # o elemento mais afastado de zero

def encode(m):
    return d*Rq(m)

def dq(x,y=0):     # distancia entre elementos de GF(q)
    x_ = lift(x-y)
    return min(x_,q-x_)

def decode(p):      # p está em Rq
    return [0 if dq(x) < dq(x,d) else 1 for x in p.list()]

In [10]:
## KeyGen

# chave privada
e = random_err()
s = random_err()

## chave publica
a = random_pol()
b = a*s + e

In [11]:
## Encrypt
m = [random.choice([0,1]) for _ in range(n)]       # mensagem aleatória

r  = random_err()
e0 = random_err()
e1 = random_err()

u = a*r + e0
v = b*r + e1 + encode(m)

In [12]:
## Decrypt

m1 = decode(v - u*s)

In [13]:
m == m1

True